In [1]:
import os
import torch
import torchaudio

from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts


In [7]:

# 파일 경로 설정
CONFIG_PATH = "./config.json"
TOKENIZER_PATH = "./vocab.json"
XTTS_CHECKPOINT = "./best_model.pth"
SPEAKER_REFERENCE = "./data/10.wav"

# 설정 로드 및 모델 초기화
config = XttsConfig()
config.load_json(CONFIG_PATH)
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False, speaker_file_path=XTTS_CHECKPOINT)
# model.cuda()

In [11]:
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])


def main(prompt):
    # output wav path
    OUTPUT_WAV_PATH = f"./{prompt}.wav"

    out = model.inference(
        prompt,
        "ko",
        gpt_cond_latent,
        speaker_embedding,
        temperature=0.7,  # Add custom parameters here
    )
    torchaudio.save(OUTPUT_WAV_PATH, torch.tensor(out["wav"]).unsqueeze(0), 24000)
    print(f"Output saved to {OUTPUT_WAV_PATH}")


if __name__ == "__main__":
    main("혼자옵서예")
